# Oposum Corpus

build corpus from oposum train/dev/test files
-> tokenized, lemmatized, remove stopwords, remove tf1 items (with functions from `prep_hdf5`, i.e. the processing procedure is the same as official)

update: just tokenized, lemmatized, remove stopwords

In [1]:
import os
from nltk.corpus import stopwords
from tqdm import tqdm
from gensim.models import Word2Vec, KeyedVectors
from time import time

%load_ext autoreload
%autoreload 2

In [2]:
%cd ../

c:\Project\group-1.3


In [3]:
import oposum_scripts.prep_hdf5 as prephdf5
from scripts.utils import *

[nltk_data] Downloading package words to C:\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
all_domains = ['bags_and_cases', 'bluetooth', 'boots', 'keyboards', 'tv', 'vacuums']
train_file_folder = './LeverageJustAFewKeywords/data/train/'
dev_test_file_folder = './LeverageJustAFewKeywords/data/gold/'
output_corpus_basedir = './processed/oposum/'

In [5]:
os.makedirs(output_corpus_basedir, exist_ok=True)

# debug example

In [32]:
domain = all_domains[0]
train_file = train_file_folder + domain + '.trn'
dev_file = dev_test_file_folder + domain + '-dev.asp'
test_file = dev_test_file_folder + domain + '-tst.asp'

In [6]:
def from_train_file(train_file):
    corpus = []
    paragraph = []
    with open(train_file, 'r') as f:
        first_line = True
        for line in tqdm(f):
            if not first_line:
                if len(line.strip()) != 0:
                    segs, orig, ids, total = prephdf5.line_to_words(line, min_len=0, 
                                                stop_words=set(stopwords.words('english')))
                    flatten_tokens = [t for seg in segs for t in seg]
                    paragraph.extend(flatten_tokens)
                else:   # a blank line
                    first_line = True
                    corpus.append(paragraph)
                    paragraph = []
            else:   # line with code
                first_line = False
    return corpus

In [64]:
# train_corpus = from_train_file(train_file)

In [7]:
def print_example(data, num=5):
    for i in range(num):
        print(data[i])

In [48]:
print(len(train_corpus))
print_example(train_corpus, 5)

42632
['case', 'look', 'nice', 'plenty', 'pocket', 'stuff', 'carry', 'around', "'re", 'using', 'something', 'back', 'forth', 'office', 'thinner', 'laptop', 'enough', 'padding', 'protect', 'computer', 'scratching', 'rubbing', 'bought', 'son', 'school', 'past', 'fall', 'one', 'thinner', 'laptop', 'cover', 'soon', 'dented', 'scratched', 'constant', 'rubbing', 'screen', 'touchpad', 'control', 'left', 'little', 'white', 'mark', 'display', 'blame', 'computer', 'design', 'added', 'padding', 'case', 'problem', 'plus', 'side', 'case', 'durable', 'nothing', 'yet', 'ripped', 'torn', "'s", 'easy', 'spot', 'clean']
['bought', 'one', 'month', 'back', 'toshiba', 'laptop', 'case', 'quite', 'strong', 'thou', 'lill', 'small', '<NUM>', "''", 'toshiba', 'satellite', 'side', 'bag', 'quite', 'strong', 'sturdy', 'compact', 'u', 'r', 'thinking', 'u', 'also', 'put', 'ur', 'file', 'n', 'document', 'case', 'along', 'ur', 'laptop', 'bag', 'u', 'pro', 'strong', 'good', 'quality', 'fabric', 'n', 'zipper', 'water', 

In [8]:
def from_dev_test_file(test_file):
    corpus = []
    paragraph = []
    with open(test_file, 'r') as f:
        aspects = f.readline()
        first_line = False
        for line in f:
            if not first_line:
                if len(line.strip()) != 0:
                    segment, labels = line.strip().split('\t')
                    segs, orig, ids, total = prephdf5.line_to_words(segment, min_len=0, 
                                        stop_words=set(stopwords.words('english')), lemmatize=True)
                    flatten_segs = [t for seg in segs for t in seg]
                    paragraph.extend(flatten_segs)
                else:
                    first_line = True
                    if len(paragraph) > 0:
                        corpus.append(paragraph)
                    paragraph = []
            else:
                rcode = line.strip()
                first_line = False
        return corpus

In [38]:
dev_corpus = from_dev_test_file(dev_file)
test_corpus = from_dev_test_file(test_file)

In [49]:
print(len(dev_corpus))
print_example(dev_corpus, 5)

50
['photo', 'online', 'card', 'attached', 'case', 'show', 'foam', "n't", 'foam', 'included', 'way', 'thin', 'thin', 'thin', 'case', 'would', 'protect', 'pencil', 'let', 'alone', 'heavy', 'laptop', "n't", 'waste', 'money', 'mine', 'go', 'back', 'tomorrow']
['use', 'case', 'daily', 'basis', 'used', 'five', 'month', 'strap', 'attachment', 'handle', 'broke', 'already', 'look', 'new', 'case', 'going', 'use', 'daily', 'buy', 'something', 'better', 'quality', 'look', 'nice', 'bulky', 'brake', 'due', 'poor', 'quality']
['really', 'liked', 'bag', 'room', 'power', 'cable', 'small', 'thing', 'year', 'used', 'occasionally', 'unfortunately', 'holiday', 'hand', 'strap', 'broke', 'great', 'buy', 'careful', 'strap']
['bought', 'case', 'recomendation', 'friend', 'size', 'case', 'great', 'even', 'fit', 'te', 'seat', 'smaller', 'airplane', 'need', 'lot', 'space', 'extra', 'perfect', 'size', 'padding', 'good', 'quite', 'stylish', 'enough', 'size', 'mouse', 'power', 'cord', 'fre', 'cd', "'s", 'cell', 'pho

In [50]:
print(len(test_corpus))
print_example(test_corpus, 5)

50
['purchased', 'case', 'looked', 'functional', 'surprise', 'rivet', 'popped', 'zipper', 'broke', 'two', 'week', "'s", 'use', 'never', 'disappointed', 'sampsonite', 'product', 'written', 'received', 'communication', 'return']
['bought', 'case', 'dell', 'inspiron', 'e1505', '<NUM>', '<NUM>', "''", 'screen', 'way', 'small', 'case', 'inner', 'pocket', 'actual', 'spot', 'laptop', 'slip', 'say', 'fit', '<NUM>', "''", 'screen', 'ended', 'putting', 'laptop', 'outside', 'inner', 'pocket', 'barely', 'fit', 'wish', 'manufacturer', 'noticed', 'problem', 'listed', '<NUM>', "''", 'screen']
['case', 'look', 'pretty', 'high', 'quality', "'s", 'thin', 'getting', 'laptop', 'case', 'huge', 'hassle', 'requiring', 'hand', 'lot', 'pulling', '<NUM>', '<NUM>', "''", 'widescreen', 'dell', 'dimension', '<NUM>', 'fit', 'snug', 'stated', 'getting', 'computer', 'pain', 'wife', 'trying', 'convince', 'return', "'m", 'lazy', 'think', "'ll", 'keep', 'also', 'wireless', 'mouse', 'bulge', 'case', 'thin', 'characterist

In [51]:
corpus = train_corpus + dev_corpus + dev_corpus
print(len(corpus))

42732


In [53]:
def remove_tf1(text_list):
    '''
    tokenized list of list
    '''
    model_tf1 = Word2Vec(min_count=1, sorted_vocab=1)
    model_tf1.build_vocab(text_list)
    TF1_list = []
    for token, vocab in model_tf1.wv.vocab.items():
        if vocab.count == 1:
            TF1_list.append(token)
    print(f'length of TF1 list: {len(TF1_list)}')
    print(f'removing tf1 items ...')
    text_list = [remove_wordlist(s, set(TF1_list)) for s in text_list]
    return text_list

In [56]:
corpus_wotf1 = remove_tf1(corpus)

length of TF1 list: 14998


In [9]:
def count_tokens(text_list):
    n_token_corpus = 0
    for text in text_list:
        n_token_corpus += len(text)
    return n_token_corpus

In [63]:
print(count_tokens(corpus) - count_tokens(corpus_wotf1))
print(count_tokens(corpus_wotf1))

14998
2078103


In [58]:
print_example(corpus_wotf1)

['case', 'look', 'nice', 'plenty', 'pocket', 'stuff', 'carry', 'around', "'re", 'using', 'something', 'back', 'forth', 'office', 'thinner', 'laptop', 'enough', 'padding', 'protect', 'computer', 'scratching', 'rubbing', 'bought', 'son', 'school', 'past', 'fall', 'one', 'thinner', 'laptop', 'cover', 'soon', 'dented', 'scratched', 'constant', 'rubbing', 'screen', 'touchpad', 'control', 'left', 'little', 'white', 'mark', 'display', 'blame', 'computer', 'design', 'added', 'padding', 'case', 'problem', 'plus', 'side', 'case', 'durable', 'nothing', 'yet', 'ripped', 'torn', "'s", 'easy', 'spot', 'clean']
['bought', 'one', 'month', 'back', 'toshiba', 'laptop', 'case', 'quite', 'strong', 'thou', 'small', '<NUM>', "''", 'toshiba', 'satellite', 'side', 'bag', 'quite', 'strong', 'sturdy', 'compact', 'u', 'r', 'thinking', 'u', 'also', 'put', 'ur', 'file', 'n', 'document', 'case', 'along', 'ur', 'laptop', 'bag', 'u', 'pro', 'strong', 'good', 'quality', 'fabric', 'n', 'zipper', 'water', 'proof', 'ligh

# Run

In [10]:
wotf1 = False

In [11]:
for domain in all_domains:
    print(f'processing oposum-{domain} corpus')
    t0 = time()
    train_file = train_file_folder + domain + '.trn'
    dev_file = dev_test_file_folder + domain + '-dev.asp'
    test_file = dev_test_file_folder + domain + '-tst.asp'

    print("loading files ...")
    train_corpus = from_train_file(train_file)
    dev_corpus = from_dev_test_file(dev_file)
    test_corpus = from_dev_test_file(test_file)
    corpus = train_corpus + dev_corpus + test_corpus
    print(f"data length of {domain}: \ntrain: {len(train_corpus)}\tdev: {len(dev_corpus)}\ttest: {len(test_corpus)}")
    print(f"number of tokens: {count_tokens(corpus)}")
    unique_tokens = set([t for p in corpus for t in p])
    print(f"number of unique tokens: {len(unique_tokens)}")
    
    if wotf1:
        corpus = remove_tf1(corpus)
        pickle_save(corpus, output_corpus_basedir + domain + '_corpus_wotf1.pkl')
    else:
        pickle_save(corpus, output_corpus_basedir + domain + '_corpus.pkl')
    print(f"finished {domain} corpus processing in {time() - t0:.2f} seconds!\n\n")

processing oposum-bags_and_cases corpus
loading files ...


338439it [02:08, 2624.47it/s]


data length of bags_and_cases: 
train: 42632	dev: 50 	 test: 50
number of tokens: 2093265
number of unique tokens: 30430


241it [00:00, 2387.49it/s]

finished bags_and_cases corpus processing in 131.24 seconds!


processing oposum-bluetooth corpus
loading files ...


...

data length of tv: 
train: 56410	dev: 50 	 test: 50
number of tokens: 5204820
number of unique tokens: 59077


251it [00:00, 2486.20it/s]

finished tv corpus processing in 284.56 seconds!


processing oposum-vacuums corpus
loading files ...


751318it [04:37, 2709.01it/s]


data length of vacuums: 
train: 68166	dev: 50 	 test: 50
number of tokens: 5007647
number of unique tokens: 46243
finished vacuums corpus processing in 281.79 seconds!




# MISC

In [40]:
from nltk.stem.wordnet import WordNetLemmatizer

In [41]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('liked')

'liked'